In [7]:
from transformers import AutoTokenizer,AutoModelForCausalLM
from peft import PeftModel

In [9]:
###load model
tokenizer = AutoTokenizer.from_pretrained("qw_0.5_instruct")
model = AutoModelForCausalLM.from_pretrained("qw_0.5_instruct")
p_model = PeftModel.from_pretrained(model,model_id="./lora_qw2.5/checkpoint-276")

In [ ]:
def predict(messages, model, tokenizer):
    device = "cuda"  # Define device as CUDA
    model.to(device)  # Make sure the model is also moved to the CUDA device

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(input_ids=model_inputs.input_ids, max_new_tokens=512)

    generated_ids = [
        output_ids[len(input_ids):] 
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

INSTRUCTION = "你是一个物流专家，请根据用户的问题给出专业的回答"
questions = "2023年快递业务收入占邮政行业总收入的比重是多少，比去年增长了多少？"
messages = [
                {"role": "system", "content": f"{INSTRUCTION}"},
                {"role": "user", "content": f"{questions}"},
            ]

response = predict(messages, p_model, tokenizer)
print(response)


喜欢。
